In [2]:
import pandas as pd

# Define the file path
file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Final_dataset_updated.xlsx"

# Load the Excel file
xls = pd.ExcelFile(file_path)

# Get sheet names
sheet_names = xls.sheet_names
sheet_names


['CSK',
 'DC',
 'GT',
 'KKR',
 'LSG',
 'MI',
 'PBKS',
 'RCB',
 'RR',
 'SRH',
 'Wicket-Keeper']

In [3]:
# Load the first sheet (CSK) to inspect its structure
df_csk = pd.read_excel(xls, sheet_name="CSK")

# Display the first few rows
df_csk.head()


,Player,Span,Position,Type,Consistency,Form,Team,Nationality
0,R Ravindra,2024-2024,2,AR,22.961332,25.420023,CSK,Foreginer
1,DP Conway,2022-2023,2,BAT,153.908300,120.680042,CSK,Foreginer
2,SM Curran,2019-2024,5,AR,85.724403,32.909673,CSK,Foreginer
3,J Overton,2023-2025,6,AR,28.007414,21.414449,CSK,Foreginer
4,NT Ellis,2021-2024,8,BOWL,-1.992150,-3.584018,CSK,Foreginer


## Main Squad Selection Code Best Playing 11

In [55]:
import pandas as pd

def load_data(file_path):
    """Load the dataset from an Excel file."""
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names
    dfs = {sheet: pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_names}
    return dfs

def preprocess_data(dfs):
    """
    For non-wicketkeeper sheets, compute:
      Weighted_Score = 0.3 * Consistency + 0.7 * Form.
    For the Wicket-Keeper sheet, rename 'Consistency_Score' to 'Consistency'
      and compute Weighted_Score similarly.
    """
    for team, df in dfs.items():
        if team != "Wicket-Keeper":
            df["Weighted_Score"] = (0.3 * df["Consistency"]) + (0.7 * df["Form"])
    df_wk = dfs["Wicket-Keeper"]
    df_wk.rename(columns={"Consistency_Score": "Consistency"}, inplace=True)
    df_wk["Weighted_Score"] = (0.3 * df_wk["Consistency"]) + (0.7 * df_wk["Form"])
    return dfs, df_wk

def select_best_players_by_position(df_team):
    """
    For each position from 1 to 11, compare all players in that position and pick
    the candidate with the highest Weighted_Score while ensuring that no more than
    4 'Foreginer' players are selected in total.
    
    If no candidate is found for a given position (under the foreigner constraint),
    a warning is printed and the best candidate ignoring the constraint is selected.
    
    Returns a list of the selected player rows.
    """
    best_players = []
    foreign_count = 0
    
    for pos in range(1, 12):
        candidates = df_team[df_team["Position"] == pos].sort_values(by="Weighted_Score", ascending=False)
        if candidates.empty:
            print(f"Warning: No candidate available for position {pos}.")
            continue
        
        selected = None
        # Try to pick the best candidate that satisfies the foreigner constraint.
        for idx, candidate in candidates.iterrows():
            # Check if candidate is foreigner.
            if str(candidate.get("Nationality", "")).strip().lower() == "foreginer":
                if foreign_count < 4:
                    selected = candidate
                    foreign_count += 1
                    break
                else:
                    # Already reached limit; skip this candidate.
                    continue
            else:
                # Candidate is not foreigner; select immediately.
                selected = candidate
                break
        
        # If no candidate was selected due to constraint (e.g. all are foreigners),
        # fall back to the top candidate ignoring the constraint.
        if selected is None:
            selected = candidates.iloc[0]
            if str(selected.get("Nationality", "")).strip().lower() == "foreginer":
                foreign_count += 1
            print(f"Warning: For position {pos}, all available candidates are foreigners. Selecting the best candidate regardless.")
        
        best_players.append(selected)
    
    return best_players

# Example usage:
file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
dfs = load_data(file_path)
dfs, df_wk = preprocess_data(dfs)

# Select best players by position for team "CSK" with foreigner constraint enforced
df_team = dfs["RR"]  
best_players = select_best_players_by_position(df_team)

print("Best players by dataset Position with at most 4 foreigners:")
# Sorting the output by the dataset's Position
for player in sorted(best_players, key=lambda x: x["Position"]):
    pos = player["Position"]
    bowler_type = player.get("Bowler_Type", "")
    print(f"Position: {pos}, {player['Player']} - {player['Type']} {bowler_type}")


Best players by dataset Position with at most 4 foreigners:
Position: 1, YBK Jaiswal - BAT nan
Position: 2, SV Samson - BAT nan
Position: 3, N Rana - AR Spin
Position: 4, R Parag - AR Spin
Position: 5, SO Hetmyer - BAT nan
Position: 6, DC Jurel - BAT nan
Position: 7, PW Hasaranga - BOWL Spin
Position: 8, JC Archer - BOWL Pace
Position: 9, A Madhwal - BOWL Pace
Position: 10, M Theekshana - BOWL Spin
Position: 11, Sandeep Sharma - BOWL Pace


## Main Squad Selection Code Best Playing 11

In [81]:
import pandas as pd

def load_data(file_path):
    """Load the dataset from an Excel file."""
    xls = pd.ExcelFile(file_path)
    sheet_names = xls.sheet_names
    dfs = {sheet: pd.read_excel(xls, sheet_name=sheet) for sheet in sheet_names}
    return dfs

def preprocess_data(dfs):
    """
    For non-wicketkeeper sheets, compute:
      Weighted_Score = 0.3 * Consistency + 0.7 * Form.
    For the Wicket-Keeper sheet, rename 'Consistency_Score' to 'Consistency'
      and compute Weighted_Score similarly.
    """
    for team, df in dfs.items():
        if team != "Wicket-Keeper":
            df["Weighted_Score"] = (0.3 * df["Consistency"]) + (0.7 * df["Form"])
    df_wk = dfs["Wicket-Keeper"]
    df_wk.rename(columns={"Consistency_Score": "Consistency"}, inplace=True)
    df_wk["Weighted_Score"] = (0.3 * df_wk["Consistency"]) + (0.7 * df_wk["Form"])
    return dfs, df_wk

def select_best_players_by_position(df_team):
    """
    For each position from 1 to 11, compare all players in that position and pick
    the candidate with the highest Weighted_Score while ensuring that no more than
    4 'Foreginer' players are selected in total.
    
    If no candidate is found for a given position (under the foreigner constraint),
    a warning is printed and the best candidate ignoring the constraint is selected.
    
    Returns a list of the selected player rows.
    """
    best_players = []
    foreign_count = 0
    
    for pos in range(1, 12):
        candidates = df_team[df_team["Position"] == pos].sort_values(by="Weighted_Score", ascending=False)
        if candidates.empty:
            print(f"Warning: No candidate available for position {pos}.")
            continue
        
        selected = None
        # Try to pick the best candidate that satisfies the foreigner constraint.
        for idx, candidate in candidates.iterrows():
            # Check if candidate is foreigner.
            if str(candidate.get("Nationality", "")).strip().lower() == "foreginer":
                if foreign_count < 4:
                    selected = candidate
                    foreign_count += 1
                    break
                else:
                    # Already reached limit; skip this candidate.
                    continue
            else:
                # Candidate is not foreigner; select immediately.
                selected = candidate
                break
        
        # If no candidate was selected due to constraint (e.g. all are foreigners),
        # fall back to the top candidate ignoring the constraint.
        if selected is None:
            selected = candidates.iloc[0]
            if str(selected.get("Nationality", "")).strip().lower() == "foreginer":
                foreign_count += 1
            print(f"Warning: For position {pos}, all available candidates are foreigners. Selecting the best candidate regardless.")
        
        best_players.append(selected)
    
    return best_players

# Example usage:
file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
dfs = load_data(file_path)
dfs, df_wk = preprocess_data(dfs)

# Select best players by position for team "RR" with foreigner constraint enforced
df_team = dfs["SRH"]  
best_players = select_best_players_by_position(df_team)

print("Best players by dataset Position with at most 4 foreigners:")
# Sorting the output by the dataset's Position
for player in sorted(best_players, key=lambda x: x["Position"]):
    pos = player["Position"]
    bowler_type = player.get("Bowler_Type", "")
    nationality = player.get("Nationality", "N/A")
    print(f"Position: {pos}, {player['Player']} - {player['Type']} {bowler_type} - Nationality: {nationality}")

# --- Additional code to print counts of Indian and Foreigner players ---
indian_count = 0
foreigner_count = 0

for player in best_players:
    # Check the player's nationality (assuming "foreginer" indicates a foreigner)
    if str(player.get("Nationality", "")).strip().lower() == "foreginer":
        foreigner_count += 1
    else:
        indian_count += 1

print("\nPlayer nationality counts:")
print(f"Indian players: {indian_count}")
print(f"Foreigner players: {foreigner_count}")


Best players by dataset Position with at most 4 foreigners:
Position: 1, Abhishek Sharma - AR Spin - Nationality: Indian
Position: 2, TM Head - BAT nan - Nationality: Foreginer
Position: 3, Ishan Kishan - BAT nan - Nationality: Indian
Position: 4, K Nitish Kumar Reddy - AR Pace - Nationality: Indian
Position: 5, H Klaasen - BAT nan - Nationality: Foreginer
Position: 6, A Manohar - BAT nan - Nationality: Indian
Position: 7, BA Carse - AR Pace - Nationality: Foreginer
Position: 8, PJ Cummins - AR Pace - Nationality: Foreginer
Position: 9, HV Patel - AR Pace - Nationality: Indian
Position: 10, Mohammed Shami - BOWL Pace - Nationality: Indian
Position: 11, RD Chahar - BOWL Spin - Nationality: Indian

Player nationality counts:
Indian players: 7
Foreigner players: 4


In [119]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
sheet_name = "CSK"

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form + 0.3 * Consistency
df['Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']

# Display players with calculated scores (optional)
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Score']])
print("\n")

# Group players by Position (positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - Exactly 1 wicket-keeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count == 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and 1 wicket-keeper):", len(valid_squads))
print("\n")

# Optionally, display a few sample valid squads with their total scores
print("Sample Valid Squad Combinations:\n")
for idx, squad in enumerate(valid_squads[:5], start=1):
    squad_score = sum(player['Score'] for player in squad)
    print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
    for player in sorted(squad, key=lambda p: p['Position']):
        print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Nationality: {player['Nationality']}")
    print()


Players with calculated scores:
              Player  Position        Form  Consistency       Score
0         RD Gaikwad         1  109.460887   344.274000  179.904821
1          DP Conway         2  120.680042   153.908300  130.648520
2         R Ravindra         3   25.420023    22.961332   24.682416
3          V Shankar         3    3.851820     4.837989    4.147671
4        RA Tripathi         4   52.947437   322.645800  133.856946
5             S Dube         5   42.019424   110.459608   62.551479
6           DJ Hooda         5   17.656378   106.662426   44.358192
7             V Bedi         5    0.000000     0.000000    0.000000
8          SM Curran         6   32.909673    85.724403   48.754092
9          J Overton         6   21.414449    28.007414   23.392338
10          MS Dhoni         6   72.534903   730.304500  269.865782
11          RS Ghosh         7    6.214047    -1.873326    3.787835
12         RA Jadeja         7   34.203140   256.421976  100.868791
13        SK Ras

In [134]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
sheet_name = "SRH"

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form + 0.3 * Consistency
df['Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - Exactly 1 wicket-keeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count == 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and 1 wicket-keeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
# Calculate total squad score as the sum of each player's Score.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

# Print the top 5 squads with squad details and count of Indian/Foreigner players.
print("Top 5 Squads (by total squad score):\n")
for idx, squad in enumerate(top_squads, start=1):
    squad_score = sum(player['Score'] for player in squad)
    # Count nationality occurrences
    indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    
    print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
    for player in sorted(squad, key=lambda p: p['Position']):
        print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
    print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
                  Player  Position        Form  Consistency       Score  \
0        Abhishek Sharma         1   51.931370   106.121999   68.188559   
1                TM Head         2  116.981217   137.426600  123.114832   
2                A Taide         2   42.666148    61.802540   48.407066   
3           Ishan Kishan         3   73.172593   375.057300  163.738005   
4   K Nitish Kumar Reddy         4   32.626052    29.792531   31.775995   
5              H Klaasen         5   99.333317   166.108100  119.365752   
6            Sachin Baby         5   41.745531   193.586500   87.297822   
7              A Manohar         6   12.739515    55.361600   25.526140   
8               BA Carse         8   10.983610    15.875320   12.451123   
9             PJ Cummins         7   27.279144    64.783035   38.530312   
10              HV Patel         9   11.806267    62.066375   26.884300   
11        Zeeshan Ansari         9   -9.083030   -14.046000  -10.571

In [136]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
sheet_name = "RCB"  # Change to "RCB" or other sheet as needed

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form + 0.3 * Consistency
df['Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Force inclusion of HV Patel if he is in the dataset.
if "HV Patel" in df['Player'].values:
    valid_squads = [squad for squad in valid_squads if any(player['Player'] == 'HV Patel' for player in squad)]
    print("After forcing inclusion of HV Patel, valid squads:", len(valid_squads))
    print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria (including HV Patel if present).")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position        Form  Consistency       Score    Type  \
0   LS Livingstone         4   21.392604    77.716706   38.289834      AR   
1       R Shepherd         7   22.694281    20.728633   22.104586      AR   
2      MS Bhandage         6    7.693454    16.143654   10.228514      AR   
3        KH Pandya         7   22.823990   145.633904   59.666964      AR   
4    Swapnil Singh         8   20.610524    12.807622   18.269653      AR   
5          PD Salt         1   97.855912   121.416700  104.924148  BAT,WK   
6         TH David         6   65.436768   120.975700   82.098447     BAT   
7       D Padikkal         3   19.237172   120.817100   49.711151     BAT   
8       SS Chikara         2    9.550107     8.686780    9.291109     BAT   
9       RM Patidar         3   88.992590   138.074400  103.717133     BAT   
10         V Kohli         2  133.657600  1087.267000  419.740420     BAT   
11       JM Sharma         5   51.119763   1

### Final Squad code 

In [1]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form + 0.3 * Consistency
df['Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position        Form  Consistency       Score    Type  \
0          PD Salt         1   97.855912   121.416700  104.924148  BAT,WK   
1       SS Chikara         2    9.550107     8.686780    9.291109     BAT   
2          V Kohli         2  133.657600  1087.267000  419.740420     BAT   
3       D Padikkal         3   19.237172   120.817100   49.711151     BAT   
4       RM Patidar         3   88.992590   138.074400  103.717133     BAT   
5   LS Livingstone         4   21.392604    77.716706   38.289834      AR   
6        JM Sharma         5   51.119763   124.918600   73.259414  BAT,WK   
7      MS Bhandage         6    7.693454    16.143654   10.228514      AR   
8         TH David         6   65.436768   120.975700   82.098447     BAT   
9       R Shepherd         7   22.694281    20.728633   22.104586      AR   
10       KH Pandya         7   22.823990   145.633904   59.666964      AR   
11   Swapnil Singh         8   20.610524    

In [2]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
sheet_name = "CSK"  # Change to "RCB" or other sheet as needed

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate each player's score using the weighted sum (MCDM approach)
# Weight: 70% for Form and 30% for Consistency.
df['Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# ------------------------------
# MCDM: Weighted Sum Model for Squad Ranking
# ------------------------------
def compute_mcdm_score(squad):
    """
    Compute the aggregated squad score using a weighted sum model.
    Here, each player's score (computed as 0.7*Form + 0.3*Consistency)
    is summed up to represent the squad's overall performance.
    """
    return sum(player['Score'] for player in squad)

# Sort valid squads by the computed MCDM score (total squad score) in descending order.
valid_squads_sorted = sorted(valid_squads, key=compute_mcdm_score, reverse=True)

# Select top 5 squads with the highest aggregated (MCDM) score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score using MCDM):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = compute_mcdm_score(squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
              Player  Position        Form  Consistency       Score    Type  \
0         RD Gaikwad         1  109.460887   344.274000  179.904821     BAT   
1          DP Conway         2  120.680042   153.908300  130.648520     BAT   
2         R Ravindra         3   25.420023    22.961332   24.682416      AR   
3          V Shankar         3    3.851820     4.837989    4.147671      AR   
4        RA Tripathi         4   52.947437   322.645800  133.856946     BAT   
5             S Dube         5   42.019424   110.459608   62.551479      AR   
6           DJ Hooda         5   17.656378   106.662426   44.358192      AR   
7             V Bedi         5    0.000000     0.000000    0.000000  BAT,WK   
8          SM Curran         6   32.909673    85.724403   48.754092      AR   
9          J Overton         6   21.414449    28.007414   23.392338      AR   
10          MS Dhoni         6   72.534903   730.304500  269.865782  BAT,WK   
11          RS Ghosh

### Final code for squad selection using MCDM

In [3]:
import pandas as pd
import numpy as np
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
sheet_name = "CSK"  # Change as needed

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Define the criteria to be used in TOPSIS
criteria = ['Form', 'Consistency']

# Extract the decision matrix from the DataFrame
decision_matrix = df[criteria].values.astype(float)

# Normalize the decision matrix using vector normalization
norm_denominator = np.sqrt((decision_matrix ** 2).sum(axis=0))
norm_matrix = decision_matrix / norm_denominator

# Define weights for the criteria (same as before: 0.7 for Form, 0.3 for Consistency)
weights = np.array([0.7, 0.3])

# Create the weighted normalized decision matrix
weighted_norm_matrix = norm_matrix * weights

# Determine the ideal (best) and anti-ideal (worst) solutions
# For beneficial criteria, ideal is max and anti-ideal is min.
ideal_solution = weighted_norm_matrix.max(axis=0)
anti_ideal_solution = weighted_norm_matrix.min(axis=0)

# Calculate the Euclidean distance from each alternative to the ideal and anti-ideal solutions
distance_to_ideal = np.sqrt(((weighted_norm_matrix - ideal_solution) ** 2).sum(axis=1))
distance_to_anti_ideal = np.sqrt(((weighted_norm_matrix - anti_ideal_solution) ** 2).sum(axis=1))

# Compute the TOPSIS score (relative closeness to the ideal solution)
df['TOPSIS_Score'] = distance_to_anti_ideal / (distance_to_ideal + distance_to_anti_ideal)

# For comparison, you can also keep the weighted sum score if desired.
df['Weighted_Sum_Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']

# Display players with calculated TOPSIS and weighted sum scores
print("Players with TOPSIS and Weighted Sum Scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Weighted_Sum_Score', 'TOPSIS_Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Now, sort valid squads by total squad TOPSIS score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['TOPSIS_Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total TOPSIS score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    print("Top 5 Squads (by total TOPSIS score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['TOPSIS_Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total TOPSIS Score: {squad_score:.4f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - TOPSIS Score: {player['TOPSIS_Score']:.4f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with TOPSIS and Weighted Sum Scores:
              Player  Position        Form  Consistency  Weighted_Sum_Score  \
0         RD Gaikwad         1  109.460887   344.274000          179.904821   
1          DP Conway         2  120.680042   153.908300          130.648520   
2         R Ravindra         3   25.420023    22.961332           24.682416   
3          V Shankar         3    3.851820     4.837989            4.147671   
4        RA Tripathi         4   52.947437   322.645800          133.856946   
5             S Dube         5   42.019424   110.459608           62.551479   
6           DJ Hooda         5   17.656378   106.662426           44.358192   
7             V Bedi         5    0.000000     0.000000            0.000000   
8          SM Curran         6   32.909673    85.724403           48.754092   
9          J Overton         6   21.414449    28.007414           23.392338   
10          MS Dhoni         6   72.534903   730.304500          269.865782   
11     

In [4]:
import pandas as pd
import itertools

# Input and output file paths
input_excel_file = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
output_excel_file = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Output_squads.xlsx"

# Read all sheets from the Excel file into a dictionary {sheet_name: DataFrame}
sheets_dict = pd.read_excel(input_excel_file, sheet_name=None)

# Dictionary to hold output DataFrames for each sheet
output_dfs = {}

# Process each sheet individually
for sheet_name, df in sheets_dict.items():
    # Calculate score for each player: Score = 0.7 * Form + 0.3 * Consistency
    df['Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']
    
    # Define positions 1 to 11
    positions = range(1, 12)
    grouped = {}
    missing_position = False
    for pos in positions:
        players_at_pos = df[df['Position'] == pos].to_dict('records')
        if not players_at_pos:
            missing_position = True
            break
        grouped[pos] = players_at_pos
    
    # If any required position is missing, record a message and continue to next sheet.
    if missing_position:
        output_dfs[sheet_name] = pd.DataFrame({
            "Message": [f"Missing players for some positions. Cannot form a complete squad in sheet '{sheet_name}'."]
        })
        continue
    
    # Generate all possible squad combinations (one player per position)
    all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
    
    # Filter squads based on constraints:
    # - Exactly 4 foreign players (Nationality == "Foreginer")
    # - At least one wicketkeeper (i.e., 'WK' in the "Type" column)
    valid_squads = []
    for squad in all_squads:
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        wk_count = sum(1 for player in squad if 'WK' in player['Type'])
        if foreign_count == 4 and wk_count >= 1:
            valid_squads.append(squad)
            
    if not valid_squads:
        output_dfs[sheet_name] = pd.DataFrame({
            "Message": [f"No valid squads found meeting criteria in sheet '{sheet_name}'."]
        })
        continue

    # Sort valid squads by total squad score in descending order and select the top 5 squads.
    valid_squads_sorted = sorted(valid_squads,
                                 key=lambda squad: sum(player['Score'] for player in squad),
                                 reverse=True)
    top_squads = valid_squads_sorted[:5]
    
    # Prepare a list of rows containing squad and player details
    rows = []
    for squad_idx, squad in enumerate(top_squads, start=1):
        squad_total_score = sum(player['Score'] for player in squad)
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        # Sort players by their position for clarity
        for player in sorted(squad, key=lambda p: p['Position']):
            row = {
                'Squad_Number': squad_idx,
                'Squad_Total_Score': squad_total_score,
                'Position': player['Position'],
                'Player': player['Player'],
                'Form': player['Form'],
                'Consistency': player['Consistency'],
                'Player_Score': player['Score'],
                'Type': player['Type'],
                'Bowler_Type': player['Bowler_Type'],
                'Nationality': player['Nationality'],
                'Indian_Count': indian_count,
                'Foreigner_Count': foreign_count
            }
            rows.append(row)
    
    # Convert rows into a DataFrame and sort by Squad_Number (ensuring grouping)
    out_df = pd.DataFrame(rows)
    out_df.sort_values("Squad_Number", inplace=True)
    output_dfs[sheet_name] = out_df

# Write all output DataFrames to a new Excel file with each sheet retaining its own data.
# Also, apply a unique background color for each squad group if the "Squad_Number" column exists.
with pd.ExcelWriter(output_excel_file, engine='xlsxwriter') as writer:
    # Define a list of colors for squads (colors will repeat if more than the list length)
    squad_colors = ['#FFC7CE', '#C6EFCE', '#FFEB9C', '#BDD7EE', '#D9D2E9', '#FCE4D6', '#E2EFDA', '#D9E1F2']
    
    for sheet, out_df in output_dfs.items():
        out_df.to_excel(writer, sheet_name=sheet, index=False)
        workbook = writer.book
        worksheet = writer.sheets[sheet]
        
        # Only apply formatting if "Squad_Number" exists
        if "Squad_Number" in out_df.columns:
            for squad_num, group in out_df.groupby('Squad_Number'):
                color = squad_colors[(int(squad_num) - 1) % len(squad_colors)]
                cell_format = workbook.add_format({'bg_color': color})
                # Apply the format to each row in this group
                for df_index in group.index:
                    excel_row = df_index + 1  # Adjust for header row
                    worksheet.set_row(excel_row, None, cell_format)

print(f"Output with colored squads written to {output_excel_file}")


Output with colored squads written to /Users/pavanbandaru/Downloads/cricket-squad-selection/Output_squads.xlsx


In [5]:
import pandas as pd
import itertools

# Input and output file paths
input_excel_file = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Copy of Final_dataset_updated(1).xlsx"
output_excel_file = "/Users/pavanbandaru/Downloads/cricket-squad-selection/Output_squads_final.xlsx"

# Read all sheets from the Excel file into a dictionary {sheet_name: DataFrame}
sheets_dict = pd.read_excel(input_excel_file, sheet_name=None)

# Dictionary to hold output DataFrames for each sheet
output_dfs = {}

# Process each sheet individually
for sheet_name, df in sheets_dict.items():
    # Calculate score for each player: Score = 0.7 * Form + 0.3 * Consistency
    df['Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']
    
    # Define positions 1 to 11
    positions = range(1, 12)
    grouped = {}
    missing_position = False
    for pos in positions:
        players_at_pos = df[df['Position'] == pos].to_dict('records')
        if not players_at_pos:
            missing_position = True
            break
        grouped[pos] = players_at_pos
    
    # If any required position is missing, record a message and continue to next sheet.
    if missing_position:
        output_dfs[sheet_name] = pd.DataFrame({
            "Message": [f"Missing players for some positions. Cannot form a complete squad in sheet '{sheet_name}'."]
        })
        continue
    
    # Generate all possible squad combinations (one player per position)
    all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
    
    # Filter squads based on constraints:
    # - Exactly 4 foreign players (Nationality == "Foreginer")
    # - At least one wicketkeeper (i.e., 'WK' in the "Type" column)
    valid_squads = []
    for squad in all_squads:
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        wk_count = sum(1 for player in squad if 'WK' in player['Type'])
        if foreign_count == 4 and wk_count >= 1:
            valid_squads.append(squad)
            
    if not valid_squads:
        output_dfs[sheet_name] = pd.DataFrame({
            "Message": [f"No valid squads found meeting criteria in sheet '{sheet_name}'."]
        })
        continue

    # Sort valid squads by total squad score in descending order and select the top 5 squads.
    valid_squads_sorted = sorted(valid_squads,
                                 key=lambda squad: sum(player['Score'] for player in squad),
                                 reverse=True)
    top_squads = valid_squads_sorted[:5]
    
    # Prepare a list of rows containing squad and player details
    rows = []
    for squad_idx, squad in enumerate(top_squads, start=1):
        squad_total_score = sum(player['Score'] for player in squad)
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        # Sort players by their position for clarity
        for player in sorted(squad, key=lambda p: p['Position']):
            row = {
                'Squad_Number': squad_idx,
                'Squad_Total_Score': squad_total_score,
                'Position': player['Position'],
                'Player': player['Player'],
                'Form': player['Form'],
                'Consistency': player['Consistency'],
                'Player_Score': player['Score'],
                'Type': player['Type'],
                'Bowler_Type': player['Bowler_Type'],
                'Nationality': player['Nationality'],
                'Indian_Count': indian_count,
                'Foreigner_Count': foreign_count
            }
            rows.append(row)
    
    # Convert rows into a DataFrame and sort by both Squad_Number and Position
    out_df = pd.DataFrame(rows)
    out_df.sort_values(["Squad_Number", "Position"], inplace=True)
    output_dfs[sheet_name] = out_df

# Write all output DataFrames to a new Excel file with each sheet retaining its own data.
# Also, apply a unique background color for each squad group if the "Squad_Number" column exists.
with pd.ExcelWriter(output_excel_file, engine='xlsxwriter') as writer:
    # Define a list of colors for squads (colors will repeat if more than the list length)
    squad_colors = ['#FFC7CE', '#C6EFCE', '#FFEB9C', '#BDD7EE', '#D9D2E9', '#FCE4D6', '#E2EFDA', '#D9E1F2']
    
    for sheet, out_df in output_dfs.items():
        out_df.to_excel(writer, sheet_name=sheet, index=False)
        workbook = writer.book
        worksheet = writer.sheets[sheet]
        
        # Only apply formatting if "Squad_Number" exists
        if "Squad_Number" in out_df.columns:
            for squad_num, group in out_df.groupby('Squad_Number'):
                color = squad_colors[(int(squad_num) - 1) % len(squad_colors)]
                cell_format = workbook.add_format({'bg_color': color})
                # Apply the format to each row in this group
                for df_index in group.index:
                    excel_row = df_index + 1  # Adjust for header row
                    worksheet.set_row(excel_row, None, cell_format)

print(f"Output with colored squads written to {output_excel_file}")


Output with colored squads written to /Users/pavanbandaru/Downloads/cricket-squad-selection/Output_squads_final.xlsx


## Final squad code for consistency alone

In [2]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one.xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form + 0.3 * Consistency
df['Score'] = 0.0 * df['Form'] + 1 * df['Consistency']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position        Form  Consistency        Score    Type  \
0          PD Salt         1   97.855912   121.416700   121.416700  BAT,WK   
1          V Kohli         2  133.657600  1087.267000  1087.267000     BAT   
2       SS Chikara         2    9.550107     8.686780     8.686780     BAT   
3       RM Patidar         3   88.992590   138.074400   138.074400     BAT   
4       D Padikkal         3   19.237172   120.817100   120.817100     BAT   
5   LS Livingstone         4   21.392604    77.716706    77.716706      AR   
6        JM Sharma         5   51.119763   124.918600   124.918600  BAT,WK   
7      MS Bhandage         6    7.693454    16.143654    16.143654      AR   
8         TH David         6   65.436768   120.975700   120.975700     BAT   
9        KH Pandya         7   22.823990   145.633904   145.633904      AR   
10      R Shepherd         7   22.694281    20.728633    20.728633      AR   
11   Swapnil Singh         8   2

## Final squad code for form alone

In [3]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one.xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form + 0.3 * Consistency
df['Score'] = 1 * df['Form'] + 0.0 * df['Consistency']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position        Form  Consistency       Score    Type  \
0          PD Salt         1   97.855912   121.416700   97.855912  BAT,WK   
1          V Kohli         2  133.657600  1087.267000  133.657600     BAT   
2       SS Chikara         2    9.550107     8.686780    9.550107     BAT   
3       RM Patidar         3   88.992590   138.074400   88.992590     BAT   
4       D Padikkal         3   19.237172   120.817100   19.237172     BAT   
5   LS Livingstone         4   21.392604    77.716706   21.392604      AR   
6        JM Sharma         5   51.119763   124.918600   51.119763  BAT,WK   
7      MS Bhandage         6    7.693454    16.143654    7.693454      AR   
8         TH David         6   65.436768   120.975700   65.436768     BAT   
9        KH Pandya         7   22.823990   145.633904   22.823990      AR   
10      R Shepherd         7   22.694281    20.728633   22.694281      AR   
11   Swapnil Singh         8   20.610524    

## Final squad code for both consistency and form

In [ ]:
import pandas as pd
import itertools

excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one.xlsx"
sheet_name = "RCB"

df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

df['Score'] = 0.7 * df['Form'] + 0.3 * df['Consistency']

print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form', 'Consistency', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position        Form  Consistency       Score    Type  \
0          PD Salt         1   97.855912   121.416700  104.924148  BAT,WK   
1          V Kohli         2  133.657600  1087.267000  419.740420     BAT   
2       SS Chikara         2    9.550107     8.686780    9.291109     BAT   
3       RM Patidar         3   88.992590   138.074400  103.717133     BAT   
4       D Padikkal         3   19.237172   120.817100   49.711151     BAT   
5   LS Livingstone         4   21.392604    77.716706   38.289834      AR   
6        JM Sharma         5   51.119763   124.918600   73.259414  BAT,WK   
7      MS Bhandage         6    7.693454    16.143654   10.228514      AR   
8         TH David         6   65.436768   120.975700   82.098447     BAT   
9        KH Pandya         7   22.823990   145.633904   59.666964      AR   
10      R Shepherd         7   22.694281    20.728633   22.104586      AR   
11   Swapnil Singh         8   20.610524    

## Final squad code for both consistency and form using AHP

In [1]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one copy.xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form_AHP + 0.3 * Consistency_AHP
df['Score'] = 0.7 * df['Form_AHP'] + 0.3 * df['Consistency_AHP']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form_AHP', 'Consistency_AHP', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position   Form_AHP  Consistency_AHP       Score    Type  \
0          PD Salt         1  56.787230        67.052939   59.866943  BAT,WK   
1          V Kohli         2  70.983470       452.085246  185.314003     BAT   
2       SS Chikara         2  10.556400        10.556397   10.556399     BAT   
3       RM Patidar         3  52.286320        71.323661   57.997522     BAT   
4       D Padikkal         4  16.040340       104.713407   42.642260     BAT   
5   LS Livingstone         4  15.777390       101.354033   41.450383      AR   
6        JM Sharma         5  32.600720        65.363815   42.429649  BAT,WK   
7      MS Bhandage         6   9.752964        16.925261   11.904653      AR   
8         TH David         6  41.507540        66.978910   49.148951     BAT   
9        KH Pandya         7  17.326540        71.179393   33.482396      AR   
10      R Shepherd         7  22.528510        18.194121   21.228193      AR   
11   Swa

## Final code for Form alone using AHP

In [2]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one copy.xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form_AHP + 0.3 * Consistency_AHP
df['Score'] = 1 * df['Form_AHP'] + 0 * df['Consistency_AHP']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form_AHP', 'Consistency_AHP', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position   Form_AHP  Consistency_AHP      Score    Type  \
0          PD Salt         1  56.787230        67.052939  56.787230  BAT,WK   
1          V Kohli         2  70.983470       452.085246  70.983470     BAT   
2       SS Chikara         2  10.556400        10.556397  10.556400     BAT   
3       RM Patidar         3  52.286320        71.323661  52.286320     BAT   
4       D Padikkal         4  16.040340       104.713407  16.040340     BAT   
5   LS Livingstone         4  15.777390       101.354033  15.777390      AR   
6        JM Sharma         5  32.600720        65.363815  32.600720  BAT,WK   
7      MS Bhandage         6   9.752964        16.925261   9.752964      AR   
8         TH David         6  41.507540        66.978910  41.507540     BAT   
9        KH Pandya         7  17.326540        71.179393  17.326540      AR   
10      R Shepherd         7  22.528510        18.194121  22.528510      AR   
11   Swapnil Singh  

## Final code for consistency alone using AHP

In [3]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one copy.xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form_AHP + 0.3 * Consistency_AHP
df['Score'] = 0 * df['Form_AHP'] + 1 * df['Consistency_AHP']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form_AHP', 'Consistency_AHP', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position   Form_AHP  Consistency_AHP       Score    Type  \
0          PD Salt         1  56.787230        67.052939   67.052939  BAT,WK   
1          V Kohli         2  70.983470       452.085246  452.085246     BAT   
2       SS Chikara         2  10.556400        10.556397   10.556397     BAT   
3       RM Patidar         3  52.286320        71.323661   71.323661     BAT   
4       D Padikkal         4  16.040340       104.713407  104.713407     BAT   
5   LS Livingstone         4  15.777390       101.354033  101.354033      AR   
6        JM Sharma         5  32.600720        65.363815   65.363815  BAT,WK   
7      MS Bhandage         6   9.752964        16.925261   16.925261      AR   
8         TH David         6  41.507540        66.978910   66.978910     BAT   
9        KH Pandya         7  17.326540        71.179393   71.179393      AR   
10      R Shepherd         7  22.528510        18.194121   18.194121      AR   
11   Swa

## Final squad code for both consistency and form using PCA

In [6]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one copy.xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form_PCA + 0.3 * Consistency_PCA
df['Score'] = 0.7 * df['Form_PCA'] + 0.3 * df['Consistency_PCA']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form_PCA', 'Consistency_PCA', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position    Form_PCA  Consistency_PCA       Score    Type  \
0          PD Salt         1  117.581370       142.532160  125.066607  BAT,WK   
1          V Kohli         2  176.382490      1606.571507  605.439195     BAT   
2       SS Chikara         2   10.508518         4.487546    8.702226     BAT   
3       RM Patidar         3  105.123880       168.145551  124.030381     BAT   
4       D Padikkal         4   16.585913       318.969067  107.300859     BAT   
5   LS Livingstone         4   10.777063       300.560097   97.711973      AR   
6        JM Sharma         5   54.841621       151.931855   83.968691  BAT,WK   
7      MS Bhandage         6    4.962275        14.628897    7.862261      AR   
8         TH David         6   71.171688       138.857622   91.477468     BAT   
9        KH Pandya         7   13.232199       110.277887   42.345906      AR   
10      R Shepherd         7    6.834181         9.933095    7.763855      AR

## Final squad code for consistency using PCA

In [7]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one copy.xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form_PCA + 0.3 * Consistency_PCA
df['Score'] = 0 * df['Form_PCA'] + 1 * df['Consistency_PCA']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form_PCA', 'Consistency_PCA', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position    Form_PCA  Consistency_PCA        Score  \
0          PD Salt         1  117.581370       142.532160   142.532160   
1          V Kohli         2  176.382490      1606.571507  1606.571507   
2       SS Chikara         2   10.508518         4.487546     4.487546   
3       RM Patidar         3  105.123880       168.145551   168.145551   
4       D Padikkal         4   16.585913       318.969067   318.969067   
5   LS Livingstone         4   10.777063       300.560097   300.560097   
6        JM Sharma         5   54.841621       151.931855   151.931855   
7      MS Bhandage         6    4.962275        14.628897    14.628897   
8         TH David         6   71.171688       138.857622   138.857622   
9        KH Pandya         7   13.232199       110.277887   110.277887   
10      R Shepherd         7    6.834181         9.933095     9.933095   
11   Swapnil Singh         8    6.977548         4.963966     4.963966   
12    

## Final squad code for form using PCA

In [8]:
import pandas as pd
import itertools

# Specify the Excel file path and sheet name directly
excel_file_path = "/Users/pavanbandaru/Downloads/cricket-squad-selection/ipl_correct_one copy.xlsx"
sheet_name = "RCB" 

# Read the Excel file and load the specified sheet into a DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name)

# Calculate score for each player: Score = 0.7 * Form_PCA + 0.3 * Consistency_PCA
df['Score'] = 1 * df['Form_PCA'] + 0 * df['Consistency_PCA']

# Optional: Display players with calculated scores
print("Players with calculated scores:")
print(df[['Player', 'Position', 'Form_PCA', 'Consistency_PCA', 'Score', 'Type', 'Nationality', 'Bowler_Type']])
print("\n")

# Group players by Position (assuming positions 1 to 11)
positions = range(1, 12)
grouped = {pos: df[df['Position'] == pos].to_dict('records')
           for pos in positions if not df[df['Position'] == pos].empty}

# Display available players per position for verification
for pos in positions:
    count = len(grouped[pos]) if pos in grouped else 0
    print(f"Position {pos}: {count} player(s)")
print("\n")

# Generate all possible squad combinations (one player per position)
all_squads = list(itertools.product(*(grouped[pos] for pos in positions)))
print("Total possible squads (before applying constraints):", len(all_squads))
print("\n")

# Filter squads to meet the constraints:
# - Exactly 4 foreign players (Nationality == "Foreginer")
# - At least one wicketkeeper (i.e. 'WK' appears in the "Type" column)
valid_squads = []
for squad in all_squads:
    foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
    wk_count = sum(1 for player in squad if 'WK' in player['Type'])
    if foreign_count == 4 and wk_count >= 1:
        valid_squads.append(squad)

print("Total valid squads (with exactly 4 foreign players and at least 1 wicketkeeper):", len(valid_squads))
print("\n")

# Sort valid squads by total squad score in descending order.
valid_squads_sorted = sorted(valid_squads,
                             key=lambda squad: sum(player['Score'] for player in squad),
                             reverse=True)

# Select top 5 squads with the highest total score.
top_squads = valid_squads_sorted[:5]

if not top_squads:
    print("No valid squads found meeting all criteria.")
else:
    # Print the top 5 squads with squad details and counts of Indian/Foreigner players.
    print("Top 5 Squads (by total squad score):\n")
    for idx, squad in enumerate(top_squads, start=1):
        squad_score = sum(player['Score'] for player in squad)
        # Count nationality occurrences
        indian_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'indian')
        foreign_count = sum(1 for player in squad if player['Nationality'].strip().lower() == 'foreginer')
        
        print(f"Squad {idx} (Total Score: {squad_score:.2f}):")
        for player in sorted(squad, key=lambda p: p['Position']):
            print(f"  Position {player['Position']}: {player['Player']} - Score: {player['Score']:.2f} | Type: {player['Type']} | Bowler Type: {player['Bowler_Type']} | Nationality: {player['Nationality']}")
        print(f"  --> Count: Indian = {indian_count}, Foreginer = {foreign_count}\n")


Players with calculated scores:
            Player  Position    Form_PCA  Consistency_PCA       Score    Type  \
0          PD Salt         1  117.581370       142.532160  117.581370  BAT,WK   
1          V Kohli         2  176.382490      1606.571507  176.382490     BAT   
2       SS Chikara         2   10.508518         4.487546   10.508518     BAT   
3       RM Patidar         3  105.123880       168.145551  105.123880     BAT   
4       D Padikkal         4   16.585913       318.969067   16.585913     BAT   
5   LS Livingstone         4   10.777063       300.560097   10.777063      AR   
6        JM Sharma         5   54.841621       151.931855   54.841621  BAT,WK   
7      MS Bhandage         6    4.962275        14.628897    4.962275      AR   
8         TH David         6   71.171688       138.857622   71.171688     BAT   
9        KH Pandya         7   13.232199       110.277887   13.232199      AR   
10      R Shepherd         7    6.834181         9.933095    6.834181      AR